# Table des distances entre les centroïdes des IRIS

A partir des données géographiques des contours des IRIS, nous allons calculer les centroïdes puis les distances entre chaqu'un. Les applications peuvent être nombreuses.

* [**I. Lecture des données**](#lecture)
* [**II. Calcul des centroïdes**](#centroide)
* [**III. Calcul de la distance Haversine entre 2 points**](#distance)
* [**IV. Création d'un dataframe avec les distances**](#dataframe_distance)

***

On trouve les données du contour des IRIS sur le site suivant : https://geoservices.ign.fr/contoursiris. Les données géographiques contenues dans ce fichier sont en Lambert 93.

In [1]:
import pandas as pd
import geopandas as gpd

<h1><a id="lecture"> Lecture des données </a></h1>

In [2]:
geoiris = gpd.read_file("geographie/CONTOURS-IRIS_2-1_SHP_LAMB93_FXX-2021/CONTOURS-IRIS.shp")

In [3]:
geoiris.head()

,INSEE_COM,NOM_COM,IRIS,CODE_IRIS,NOM_IRIS,TYP_IRIS,geometry
0,72191,Mayet,0000,721910000,Mayet,Z,"POLYGON ((498083.500 6747517.400, 498128.000 6..."
1,77248,Lesches,0000,772480000,Lesches,Z,"POLYGON ((685753.100 6868612.900, 685757.700 6..."
2,51426,Péas,0000,514260000,Péas,Z,"POLYGON ((759067.200 6849592.700, 758778.600 6..."
3,81199,Padiès,0000,811990000,Padiès,Z,"POLYGON ((651482.800 6326359.400, 651475.600 6..."
4,59225,Feignies,0102,592250102,Sud,H,"POLYGON ((767673.500 7022290.500, 767647.200 7..."


In [4]:
geoiris = geoiris.rename(columns={'geometry': 'IRIS_geometry'})

In [5]:
geoiris.dtypes

INSEE_COM          object
NOM_COM            object
IRIS               object
CODE_IRIS          object
NOM_IRIS           object
TYP_IRIS           object
IRIS_geometry    geometry
dtype: object

In [6]:
#geoiris["IRIS_geometry"][0]

In [7]:
#str(geoiris["IRIS_geometry"][0])

La ligne ci-dessous nous permettra d'effectuer des tests :

In [8]:
test = geoiris["IRIS_geometry"][0]

<h1><a id="centroide"> Calcul des centroïdes </a></h1>

In [9]:
from shapely import wkt

In [10]:
test.centroid.wkt

'POINT (496259.3244715297 6743965.879059711)'

<div class="alert alert-block alert-info">
<b>wkt :</b> est un langage de balisage de texte pour représenter des objets de géométrie vectorielle, cela permet donc de "traduire" une forme géométrique afin qu'il soit lisible par l'Homme.
</div>

Calculons les centroïdes de chaque IRIS :

In [11]:
geoiris["centroideLambert93"] = ""

In [12]:
for i in range(len(geoiris)):
    geoiris.loc[i,'centroideLambert93'] = geoiris.loc[i,'IRIS_geometry'].centroid

In [13]:
geoiris.head()

,INSEE_COM,NOM_COM,IRIS,CODE_IRIS,NOM_IRIS,TYP_IRIS,IRIS_geometry,centroideLambert93
0,72191,Mayet,0000,721910000,Mayet,Z,"POLYGON ((498083.500 6747517.400, 498128.000 6...",POINT (496259.3244715297 6743965.879059711)
1,77248,Lesches,0000,772480000,Lesches,Z,"POLYGON ((685753.100 6868612.900, 685757.700 6...",POINT (684819.2562692661 6868391.508534629)
2,51426,Péas,0000,514260000,Péas,Z,"POLYGON ((759067.200 6849592.700, 758778.600 6...",POINT (757290.6480595496 6848727.502489925)
3,81199,Padiès,0000,811990000,Padiès,Z,"POLYGON ((651482.800 6326359.400, 651475.600 6...",POINT (649120.3356224176 6327536.809572649)
4,59225,Feignies,0102,592250102,Sud,H,"POLYGON ((767673.500 7022290.500, 767647.200 7...",POINT (764793.8338451331 7021294.756690773)


<h1><a id="distance"> Calcul de la distance Haversine entre 2 points </a></h1>

Nous devons d'abord convertir les centroïdes dans le format WSG84 (système internationnal) afin d'utiliser la fonction haversine du package haversine :

In [14]:
import pyproj
from shapely.ops import transform

In [15]:
inProj = pyproj.CRS('EPSG:2154') #Lambert93
outProj = pyproj.CRS('EPSG:4326') #WSG84

project = pyproj.Transformer.from_crs(inProj, outProj, always_xy=True).transform

In [16]:
geoiris["centroideWSG84"] = ""

In [17]:
for i in range(len(geoiris)):
    utm_point = transform(project, geoiris["centroideLambert93"][i])
    geoiris.loc[i,'centroideWSG84'] = utm_point

In [18]:
geoiris.head()

,INSEE_COM,NOM_COM,IRIS,CODE_IRIS,NOM_IRIS,TYP_IRIS,IRIS_geometry,centroideLambert93,centroideWSG84
0,72191,Mayet,0000,721910000,Mayet,Z,"POLYGON ((498083.500 6747517.400, 498128.000 6...",POINT (496259.3244715297 6743965.879059711),POINT (0.2796914983779593 47.76449762404098)
1,77248,Lesches,0000,772480000,Lesches,Z,"POLYGON ((685753.100 6868612.900, 685757.700 6...",POINT (684819.2562692661 6868391.508534629),POINT (2.79287013093658 48.9153358646321)
2,51426,Péas,0000,514260000,Péas,Z,"POLYGON ((759067.200 6849592.700, 758778.600 6...",POINT (757290.6480595496 6848727.502489925),POINT (3.779017410081798 48.73612598145969)
3,81199,Padiès,0000,811990000,Padiès,Z,"POLYGON ((651482.800 6326359.400, 651475.600 6...",POINT (649120.3356224176 6327536.809572649),POINT (2.365131241705289 44.0449922815527)
4,59225,Feignies,0102,592250102,Sud,H,"POLYGON ((767673.500 7022290.500, 767647.200 7...",POINT (764793.8338451331 7021294.756690773),POINT (3.908016927184398 50.28618033140764)


***

**Test sur calcul de distance**

Nous allons prendre 2 centroïdes et calculer la distance entre ces 2 points. Cela va nous permettre de vérifier si le résultat est correct (via Google Map).


In [19]:
from haversine import haversine, Unit

In [20]:
test1 = geoiris.loc[1,'centroideWSG84']
testA = (test1.y,test1.x)
testA

(48.9153358646321, 2.7928701309365795)

In [21]:
test2 = geoiris.loc[2,'centroideWSG84']
testB = (test2.y,test2.x)
testB

(48.73612598145969, 3.779017410081798)

In [22]:
haversine(testA, testB, unit='km')

74.89054105021961

***

Création d'une fonction permettant de calculer la distance entre 2 points en km :

In [23]:
def distanceHaversine(PointA,PointB) :
    latLongA = (PointA.y,PointA.x)
    latLongB = (PointB.y,PointB.x)
    return haversine(latLongA, latLongB, unit='km')

In [24]:
distanceHaversine(geoiris.loc[1,'centroideWSG84'],geoiris.loc[2,'centroideWSG84'])

74.89054105021961

<h1><a id="dataframe_distance"> Création d'un dataframe avec les distances </a></h1>

Création d'un dataframe vide :

In [25]:
distances_IRIS = pd.DataFrame(columns=['codeIRIS1','codeIRIS2','distanceKm'])

In [26]:
distances_IRIS

,codeIRIS1,codeIRIS2,distanceKm


In [27]:
import time

In [28]:
start = time.time()
compteur=0
#for i in range(len(geoiris)):
for i in range(0,2):
    for j in range(len(geoiris)):
        # On rempli la première cellule
        distances_IRIS.loc[compteur,'codeIRIS1']= geoiris.loc[i,'CODE_IRIS']
        distances_IRIS.loc[compteur,'codeIRIS2']= geoiris.loc[j,'CODE_IRIS']
        
        distanceAB = distanceHaversine(geoiris.loc[i,'centroideWSG84'],geoiris.loc[j,'centroideWSG84'])
        distances_IRIS.loc[compteur,'distanceKm']= distanceAB
        
        compteur += 1
end = time.time()

In [29]:
print("Temps d'exécution :",(end - start)/60)

Temps d'exécution : 11.295932801564534


In [30]:
distances_IRIS

,codeIRIS1,codeIRIS2,distanceKm
0,721910000,721910000,0
1,721910000,772480000,225.548
2,721910000,514260000,280.687
3,721910000,811990000,443.909
4,721910000,592250102,385.412
...,...,...,...
97173,772480000,710730102,268.578
97174,772480000,385450101,484.016
97175,772480000,740630000,422.978
97176,772480000,951010000,85.9998
